In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
import warnings
# Suppress all warnings
warnings.filterwarnings("ignore")

In [3]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [4]:
from llama_index.llms.openai import OpenAI

# llm = OpenAI(model="gpt-3.5-turbo")
llm = OpenAI(model="gpt-4-turbo-preview")

In [5]:
papers = ['board_meeting_dec23.pdf',
         'board_meeting_nov23.pdf',
         'board_meeting_oct23.pdf']

In [6]:
from utils import get_doc_tools
from pathlib import Path

paper_to_tools_dict = {}
for paper in papers:
    print(f"Getting tools for paper: {paper}")
    vector_tool, summary_tool = get_doc_tools(paper, Path(paper).stem)
    paper_to_tools_dict[paper] = [vector_tool, summary_tool]

Getting tools for paper: board_meeting_dec23.pdf
Getting tools for paper: board_meeting_nov23.pdf
Getting tools for paper: board_meeting_oct23.pdf


In [7]:
# Flatenning the tools in list
all_tools = [t for paper in papers for t in paper_to_tools_dict[paper]]

# Agents with tool selection capability

In [8]:
# define an "object" index and retriever over these tools
from llama_index.core import VectorStoreIndex
from llama_index.core.objects import ObjectIndex

obj_index = ObjectIndex.from_objects(
    all_tools,
    index_cls=VectorStoreIndex,
)

In [9]:
#Retrive too 3 tools based on the query
obj_retriever = obj_index.as_retriever(similarity_top_k=3)

In [10]:
# example of tools retrived by a simple query
tools = obj_retriever.retrieve(
    "summarise the nov 23 board meeting notes"
)
tools[0].metadata

ToolMetadata(description='Useful for summarization questions related to board_meeting_nov23', name='summary_tool_board_meeting_nov23', fn_schema=<class 'llama_index.core.tools.types.DefaultToolFnSchema'>, return_direct=False)

In [11]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    tool_retriever=obj_retriever,
    llm=llm, 
    system_prompt=""" \
You are an agent designed to answer queries over a set of given papers.
Please always use the tools provided to answer a question. Do not rely on prior knowledge.\

""",
    verbose=True
)
agent = AgentRunner(agent_worker)

In [12]:
response = agent.query(
    "was anything discussed about atopic dermatitis in any of the board meeting"
)
# print(str(response))

Added user message to memory: was anything discussed about atopic dermatitis in any of the board meeting
=== Calling Function ===
Calling function: summary_tool_board_meeting_oct23 with args: {"input": "atopic dermatitis"}
=== Function Output ===
The College of Pharmacy's recommendations, the pipeline review report, and the provided information do not mention atopic dermatitis. Tofidence ™(Tocilizumab-bavi) is a biosimilar to Actemra®(Tocilizumab) approved for the treatment of atopic dermatitis in adults with moderate-to-severe atopic dermatitis not adequately controlled with other systemic drug products, including biologics, or when those therapies are not advisable. The unique criteria for the approval of medications for atopic dermatitis include having an FDA-approved diagnosis of atopic dermatitis, confirmation of baseline assessments and follow-up monitoring, and a trial of at least one Tier-1 medication that did not yield adequate relief of symptoms or resulted in intolerable adv

Retrying llama_index.llms.openai.base.OpenAI._achat in 0.41236773593745546 seconds as it raised RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-Xnt90aHtb4ZX1UUscgKKvCl1 on tokens per min (TPM): Limit 200000, Used 196617, Requested 14726. Please try again in 3.402s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}.


=== Function Output ===
Atopic dermatitis is a common, chronic skin condition characterized by red, itchy rashes. It often appears in early childhood and is linked to a combination of genetic and environmental factors. Treatment typically involves moisturizing the skin, avoiding triggers, and using topical corticosteroids or other medications to manage symptoms.
=== Calling Function ===
Calling function: summary_tool_board_meeting_nov23 with args: {"input": "atopic dermatitis"}


Retrying llama_index.llms.openai.base.OpenAI._achat in 0.6096044582914619 seconds as it raised RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-Xnt90aHtb4ZX1UUscgKKvCl1 on tokens per min (TPM): Limit 200000, Used 190068, Requested 13630. Please try again in 1.109s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}.
Retrying llama_index.llms.openai.base.OpenAI._achat in 0.3186400107388725 seconds as it raised RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-Xnt90aHtb4ZX1UUscgKKvCl1 on tokens per min (TPM): Limit 200000, Used 188421, Requested 15010. Please try again in 1.029s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}.
Retrying llama_index.llms.openai.base.OpenAI._achat in 0.4283168

=== Function Output ===
Encountered error: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
=== LLM Response ===
Atopic dermatitis was discussed in the context of the board meetings as follows:

- **October 2023 Meeting**: Atopic dermatitis was mentioned in relation to Tofidence™ (Tocilizumab-bavi), a biosimilar to Actemra® (Tocilizumab) approved for the treatment of adults with moderate-to-severe atopic dermatitis not adequately controlled with other systemic drug products, including biologics, or when those therapies are not advisable. The discussion included unique criteria for the approval of medications for atopic dermatitis, such as having an FDA-approved diagnosis, confirmation of baseline assessments and follow-up monitoring, a trial of at least one Tier-1 medication that did not yield adequate relief of symptoms or resulted in intolerable adverse effects in the last 90 days, and prior stabilization on Tier-2 medication documented withi

# Agents without tool selection capabaility

In [16]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    all_tools, 
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)

Retrying llama_index.llms.openai.base.OpenAI._achat in 0.13419521617871122 seconds as it raised RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-Xnt90aHtb4ZX1UUscgKKvCl1 on tokens per min (TPM): Limit 60000, Used 59180, Requested 14143. Please try again in 13.323s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}.
Retrying llama_index.llms.openai.base.OpenAI._achat in 0.16715793050082572 seconds as it raised RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-Xnt90aHtb4ZX1UUscgKKvCl1 on tokens per min (TPM): Limit 60000, Used 58529, Requested 14867. Please try again in 13.396s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}.
Retrying llama_index.llms.openai.base.OpenAI._achat in 0.6286034

In [17]:
response = agent.query(
    "what was discussed about Annual Review of Imcivree during october 2023 board meeting"
    "summarise the discussion"    
)

Added user message to memory: what was discussed about Annual Review of Imcivree during october 2023 board meetingsummarise the discussion
=== Calling Function ===
Calling function: vector_tool_board_meeting_oct23 with args: {"query": "Annual Review of Imcivree"}
=== Function Output ===
The annual review of Imcivree includes utilization details such as total claims, total members, total cost, cost per claim, claims per member, and percentage of cost. Additionally, it provides information on the demographics of members utilizing Imcivree, top prescriber specialties, and the status of prior authorization requests submitted during fiscal year 2023.
=== Calling Function ===
Calling function: summary_tool_board_meeting_oct23 with args: {"input": "The annual review of Imcivree includes utilization details such as total claims, total members, total cost, cost per claim, claims per member, and percentage of cost. Additionally, it provides information on the demographics of members utilizing Im

Retrying llama_index.llms.openai.base.OpenAI._achat in 0.5745166704174774 seconds as it raised RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-Xnt90aHtb4ZX1UUscgKKvCl1 on tokens per min (TPM): Limit 60000, Used 59340, Requested 14160. Please try again in 13.5s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}.
Retrying llama_index.llms.openai.base.OpenAI._achat in 0.38021385133120944 seconds as it raised RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-Xnt90aHtb4ZX1UUscgKKvCl1 on tokens per min (TPM): Limit 60000, Used 59308, Requested 14243. Please try again in 13.551s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}.
Retrying llama_index.llms.openai.base.OpenAI._achat in 0.2461829716

=== Function Output ===
The annual review of Imcivree for fiscal year 2023 covers utilization details like total claims, total members, total cost, cost per claim, claims per member, and the percentage of cost. It also includes information on the demographics of members using Imcivree, top prescriber specialties, and the status of prior authorization requests submitted during the fiscal year.
=== LLM Response ===
During the October 2023 board meeting, the annual review of Imcivree was discussed, focusing on utilization details such as total claims, total members, total cost, cost per claim, claims per member, and the percentage of cost. The review also provided insights into the demographics of members using Imcivree, the specialties of the top prescribers, and the status of prior authorization requests submitted throughout fiscal year 2023.


In [ ]:
# response = agent.query(
#     "was anything discussed about atopic dermatitis in any of the board meeting"
#     )

In [ ]:
# response = agent.query(
#     "was anything discussed about atopic dermatitis in any of the board meeting"
#     "give the source nodes for reference"
#     )